In [2]:
# imports
import numpy as np
import pandas as pd

In [8]:
sample=pd.read_csv('nyc-tlc/trip data/sample.csv')
print('sample:', sample.shape, sample.head())

sample: (1000, 17)    VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2018-01-01 00:21:05   2018-01-01 00:24:23                1   
1         1  2018-01-01 00:44:55   2018-01-01 01:03:05                1   
2         1  2018-01-01 00:08:26   2018-01-01 00:14:21                2   
3         1  2018-01-01 00:20:22   2018-01-01 00:52:51                1   
4         1  2018-01-01 00:09:18   2018-01-01 00:27:06                2   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            0.5           1                  N            41            24   
1            2.7           1                  N           239           140   
2            0.8           1                  N           262           141   
3           10.2           1                  N           140           257   
4            2.5           1                  N           246           239   

   payment_type  fare_amount  extra  mta_tax  tip_amoun

In [ ]:
tripdata_201801=pd.read_csv('nyc-tlc/trip data/yellow_tripdata_2018-01.csv')
print('tripdata_201801:', tripdata_201801.shape, tripdata_201801.head())